In [ ]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

dde.config.set_random_seed(1234)

Using backend: tensorflow.compat.v1

2023-08-04 02:08:05.062887: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
num_dense_layers = 5
num_dense_nodes = 20
iterations = 10000
activation = 'tanh'
initializer = 'Glorot uniform'
optimizer = 'L-BFGS'
metrics = ['l2 relative error']

In [13]:
num_train = 32
num_test = 100

In [14]:
L = 2
n = 5

In [15]:
def psi(x):
    k = (n * np.pi) / L
    normalization_constant = np.sqrt(2.0 / L)
    return normalization_constant * np.sin(k * (x + 0.5 * L))

In [16]:
domain = dde.geometry.Interval(-L / 2, L / 2)

In [17]:
data = dde.data.Function(
    domain, 
    psi, 
    num_train, 
    num_test
)

In [18]:
net = dde.nn.FNN(
    [1] + [num_dense_nodes] * num_dense_layers + [1], 
    activation, 
    initializer
)

In [19]:
model = dde.Model(data, net)
model.compile(
    optimizer, 
    metrics=metrics
)

Compiling model...
Building feed-forward neural network...
'build' took 0.099462 s

'compile' took 0.236445 s



2023-08-01 00:36:26.538157: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-01 00:36:26.538526: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-01 00:36:26.538770: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [20]:
loss_history, train_state = model.train(iterations=iterations)

Initializing variables...
Training model...

Step      Train loss    Test loss     Test metric   
0         [5.29e-01]    [5.38e-01]    [1.04e+00]    
INFO:tensorflow:Optimization terminated with:
  Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  Objective function value: 0.000001
  Number of iterations: 393
  Number of functions evaluations: 462
462       [7.77e-07]    [1.34e-06]    [1.65e-03]    

Best model at step 462:
  train loss: 7.77e-07
  test loss: 1.34e-06
  test metric: [1.65e-03]

'train' took 3.921314 s



In [ ]:
dde.saveplot(loss_history, train_state, issave=True, isplot=True)

In [ ]:
def save_loss_plot():
    loss_file = open('loss.dat', 'r')
    
    loss_file.readline()
    steps = []
    train_loss = []
    test_loss = []
    test_metric = []
    
    for line in loss_file:
        line = line.split()
        steps.append(float(line[0]))
        train_loss.append(float(line[1]))
        test_loss.append(float(line[2]))
        test_metric.append(float(line[3]))
    
    loss_file.close()
    
    plt.plot(steps, train_loss, label='Train loss')
    plt.plot(steps, test_loss, label='Test loss')
    plt.plot(steps, test_metric, label='Test metric')
    
    plt.xlabel('training epoch')
    plt.yscale('log')
    
    plt.legend()
    plt.savefig('{}-loss.png'.format(n))
    plt.show()

In [ ]:
def save_prediction_plot():
    train_file = open('train.dat', 'r')
    
    x_train = []
    y_train = []
    train_file.readline()
    
    for line in train_file:
        line = line.split()
        x_train.append(float(line[0]))
        y_train.append(float(line[1]))
    
    train_file.close()
    
    test_file = open('test.dat', 'r')
    
    x_test = []
    y_true = []
    y_pred = []
    test_file.readline()
    
    for line in test_file:
        line = line.split()
        x_test.append(float(line[0]))
        y_true.append(float(line[1]))
        y_pred.append(float(line[2]))
    
    test_file.close()
    
    plt.plot(x_train, y_train, 'o', color='black', label='Training points')
    plt.plot(x_test, y_true, '-', color='black', label='True values')
    plt.plot(x_test, y_pred, '--', color='red', label='Predicted values')
    
    plt.xlabel('x')
    plt.ylabel('$\psi_{}$(x)'.format(n))
    
    plt.legend()
    plt.savefig('{}-results.png'.format(n))
    plt.show()

In [ ]:
save_prediction_plot()

In [ ]:
save_loss_plot()

In [ ]:
!mv loss.dat $n-loss.dat
!mv test.dat $n-test.dat
!mv train.dat $n-train.dat